In [1]:
import numpy as np
from random import randint
import random

In [2]:
class EnvGrid(object):
    """
    Class representing a grid world environment
    """
    def __init__(self):
        """
        Constructor for EnvGrid class

        Initializes the environment grid, starting position, and action list
        """
        super(EnvGrid, self).__init__()

        self.grid = [
            [0, 0, 1],
            [0, -1, 0],
            [0, 0, 0]
        ]
        # Starting position
        self.y = 2
        self.x = 0

        self.actions = [
            [-1, 0], # Up
            [1, 0], #Down
            [0, -1], # Left
            [0, 1] # Right
        ]

    def reset(self):
        """
        Resets the environment to its initial state

        Returns:
            The index of the initial state in the grid
        """
        self.y = 2
        self.x = 0
        return (self.y * 3 + self.x + 1)

    def step(self, action):
        
        """
        Takes a step in the environment given an action

        Args:
            action (int): The index of the action to take [0, 1, 2, 3]

        Returns:
            A tuple containing the index of the new state and the reward for that state
        """
        self.y = max(0, min(self.y + self.actions[action][0], 2))
        self.x = max(0, min(self.x + self.actions[action][1], 2))

        return (self.y * 3 + self.x + 1), self.grid[self.y][self.x]

    def show(self):
        """
        Prints the current state of the environment grid
        """
        print("---------------------")
        y = 0
        for line in self.grid:
            x = 0
            for pt in line:
                print("%s\t" % (pt if y != self.y or x != self.x else "X"), end="")
                x += 1
            y += 1
            print("")

    def is_finished(self):
        """
        Checks if the current state is a terminal state

        Returns:
            True if the current state is a terminal state, False otherwise
        """
        return self.grid[self.y][self.x] == 1

In [3]:
def take_action(st, Q, eps):
    # Take an action
    if random.uniform(0, 1) < eps:
        action = randint(0, 3)
    else: # Or greedy action
        action = np.argmax(Q[st])
    return action

In [4]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [5]:
def display(v, l):
    if v == max(l) and v > 0:
        return color.BOLD + color.BLUE + str(round(v, 5)).ljust(10) + color.END
    elif v < 0:
        return color.BOLD + color.RED + str(round(v, 5)).ljust(10) + color.END
    else:
        return str(round(v, 10)).ljust(15)

## Manual

In [6]:
env = EnvGrid()
st = env.reset()

Q = [
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
]

for _ in range(1):
    # Reset the game
    st = env.reset()
    while not env.is_finished():
        env.show()
        at = int(input("$>"))
        # at = take_action(st, Q, 0.4)

        stp1, r = env.step(at)
        print("s", stp1)
        print("r", r)

        # Update Q function
        atp1 = take_action(stp1, Q, 0.0)
        Q[st][at] = Q[st][at] + 0.1*(r + 0.9*Q[stp1][atp1] - Q[st][at])

        st = stp1

print('S  │ ', '\t│\t'.join(a.ljust(10) for a in ['Haut', 'Bas', 'Gauche', 'Droite']))
print('─' * 95)
for s in range(1, 10):
    print(s, ' │ ', '\t│\t'.join(display(q, Q[s]) for q in Q[s]))

---------------------
0	0	1	
0	-1	0	
X	0	0	


$> 3


s 8
r 0
---------------------
0	0	1	
0	-1	0	
0	X	0	


$> 0


s 5
r -1
---------------------
0	0	1	
0	X	0	
0	0	0	


...

s 6
r 0
---------------------
0	0	1	
0	-1	X	
0	0	0	


$> 3


s 6
r 0
---------------------
0	0	1	
0	-1	X	
0	0	0	


$> 0


s 3
r 1
S  │  Haut      	│	Bas       	│	Gauche    	│	Droite    
───────────────────────────────────────────────────────────────────────────────────────────────
1  │  0              	│	0              	│	0              	│	0              
2  │  0              	│	0              	│	0              	│	0              
3  │  0              	│	0              	│	0              	│	0              
4  │  0              	│	0              	│	0              	│	0              
5  │  0              	│	0              	│	0              	│	0.0            
6  │  0.1       	│	0              	│	0              	│	0.0            
7  │  0              	│	0              	│	0              	│	0.0            
8  │  -0.1      	│	0              	│	0              	│	0              
9  │  0              	│	0              	│	0              	│	0              


## Automatique

In [7]:
env = EnvGrid()
st = env.reset()

Q = [
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
]

for _ in range(1000):
    # Reset the game
    st = env.reset()
    while not env.is_finished():
        at = take_action(st, Q, 0.4)

        stp1, r = env.step(at)

        # Update Q function
        atp1 = take_action(stp1, Q, 0.0)
        Q[st][at] = Q[st][at] + 0.1*(r + 0.9*Q[stp1][atp1] - Q[st][at])

        st = stp1

print('S  │ ', '\t│\t'.join(a.ljust(10) for a in ['Haut', 'Bas', 'Gauche', 'Droite']))
print('─' * 95)
for s in range(1, 10):
    print(s, ' │ ', '\t│\t'.join(display(q, Q[s]) for q in Q[s]))

S  │  Haut      	│	Bas       	│	Gauche    	│	Droite    
───────────────────────────────────────────────────────────────────────────────────────────────
1  │  0.8099996648   	│	0.7289986744   	│	0.809998279    	│	0.9       
2  │  0.899997296    	│	-0.19     	│	0.8099977884   	│	1.0       
3  │  0              	│	0              	│	0              	│	0              
4  │  0.81      	│	0.656099794    	│	0.7289995338   	│	-0.19     
5  │  0.9       	│	0.5698937736   	│	0.6899646662   	│	0.7708647676   
6  │  0.99427   	│	0.1465617595   	│	-0.09039  	│	0.2181532831   
7  │  0.729     	│	0.6560999484   	│	0.6560998985   	│	0.5904899121   
8  │  -0.18309  	│	0.5492748246   	│	0.6561    	│	0.5981345405   
9  │  0.81588   	│	0.271690413    	│	0.2029829493   	│	0.3758775497   


### Explenation
we can see the best action to do in each state: exemple in the first state (top left case 0, 0) we have to go to the right in order to try to win.
There are also four negative values. These correspond to the actions leading to the negative reward from one of these adjacent squares